In [ ]:
import os
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool
import seaborn as sns

from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from pandas.api.types import is_numeric_dtype

In [ ]:
well_name = '1628'
mark = ''
left_boundary = [datetime.datetime(2019,1,1)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]
path_to_work_dir = f'data/{well_name}/restore_/'
chess_file_name = f'Скв. {well_name} (01.08.2018-28.02.2019).xls'
cs_data_filename = f'data/{well_name}/init_edit/{well_name}_first_edit.csv'
create_input_data = True

# 5. Восстановление дебитов.
Нужно, используя сгенерированные данные `well_name_restore_input` восстановить дебиты с помощью `postprocessor.py`
Для этого активировать флаги
* vfm_calc_option = True
* restore_q_liq_only = True

и определить папку с входными данными `dir_name_with_input_data`

In [ ]:
dir_name_with_input_data = 'restore_input_' + '\\'
input_data_file_name = well_name +'_restore_input'
dir_name_with_calculated_data = 'restore_' + '\\'
calculated_data_file_name = well_name +'_restore_1'

In [ ]:
path_to_work_dir + 'multiprocessing'


In [ ]:
multiprocessing_on = True
if multiprocessing_on == True:
    filenames_list = []
    for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + 'multiprocessing\\'):
        filenames_list.extend(filenames)
        break
    print(filenames_list)
    for i,j in enumerate(filenames_list):
        if i == 0:
            first_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        else:
            another_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
            first_result_data = first_result_data.append(another_result_data, sort = True)
            del first_result_data['d']
            first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
    first_result_data.to_csv(path_to_work_dir + calculated_data_file_name + '.csv')

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir +
                                                calculated_data_file_name +  '.csv')

Объединение входных и выходных данным модели. Сохранение результатов в `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(f'data/{well_name}/restore_input_/' + input_data_file_name +  '.csv', parse_dates = True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(f'data/{well_name}/restore_/{well_name}_calc_and_input.csv')

Построение графиков восстановления дебитов

In [ ]:
input_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = f'data/{well_name}/restore_/{well_name}_calc_and_input.html'
pw.plot_subplots(input_data_traces, plot_file_path, True)

# 6.2 Сведение данных адаптации и восстановления
Использование файлов типа `calc_and_input` в папках с адаптацией и восстновлением для формирования общего отчета

In [ ]:
adapt_data_with_input = pd.read_csv(f'data/{well_name}/adaptation_/{well_name}_calc_and_input.csv',
                                    parse_dates = True, index_col = 'Время'
                                   )
adapt_data_with_input = prep.mark_df_columns(adapt_data_with_input, 'ADAPT')
restore_data_with_input = pd.read_csv(f'data/{well_name}/restore_/{well_name}_calc_and_input.csv',
                                      parse_dates = True, index_col = 'Время'
                                     )
restore_data_with_input = prep.mark_df_columns(restore_data_with_input, 'RESTORE')
overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer')

Получение дебитов методом линейной интерполяции

In [ ]:
q_liq = overall_data[['Объемный дебит жидкости (СУ) (ADAPT)', 'Активная мощность (СУ) (ADAPT)']]
result = prep.make_gaps_and_interpolate(q_liq)
result = prep.mark_df_columns(result, 'INTERP')
overall_data = overall_data.join(result)

In [ ]:
overall_data['Активная мощность (СУ) (ADAPT)'] = overall_data['Активная мощность (СУ) (ADAPT)'] * 1000
overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'] = overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'] * 1000
overall_data['Загрузка двигателя (СУ) (ADAPT)'] = overall_data['Загрузка двигателя (СУ) (ADAPT)'] / 100
overall_data['Относительная ошибка расчетов (Q ж), %'] = postp.relative_error_perc(overall_data['Объемный дебит жидкости (СУ) (ADAPT)'],
                                                                      overall_data['Q ж, м3/сут (Модель) (RESTORE)']).abs()
overall_data['Относительная ошибка расчетов (N акт), %'] = postp.relative_error_perc(overall_data['Активная мощность (СУ) (ADAPT)'],
                                                                      overall_data['Мощность, передаваемая СУ (Модель) (RESTORE)']).abs()
overall_data['Относительная ошибка расчетов (Q ж) (INTERP), %'] = postp.relative_error_perc(overall_data['Объемный дебит жидкости (СУ) (ADAPT)'],
                                                                      overall_data['Объемный дебит жидкости (СУ) (ADAPT) (INTERP)']).abs()
overall_data['Относительная ошибка расчетов (N акт) (INTERP), %'] = postp.relative_error_perc(overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'],
                                                                      overall_data['Мощность, передаваемая СУ (Модель) (RESTORE)']).abs()

In [ ]:
liquid_rates = {'Дебиты': 
                ['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель) (ADAPT)',
                  'Объемный дебит жидкости (СУ) (ADAPT) (INTERP)', 'Q ж, м3/сут (Модель) (RESTORE)'] }
essential_mertics = {'Сравнение методов расчета':
                     ['Относительная ошибка расчетов (Q ж), %',
            'Относительная ошибка расчетов (Q ж) (INTERP), %']}

calibrs = {'Калибровки по напору и мощности':
           ['К. калибровки по напору - множитель (Модель) (ADAPT)',
               'К. калибровки по мощности - множитель (Модель) (ADAPT)', 
                  'К. калибровки по напору - множитель (Модель) (RESTORE)',
               'К. калибровки по мощности - множитель (Модель) (RESTORE)']}

gor_wc = {'ГФ, м3/м3 и Обводненность, %':
          ['ГФ (Модель) (ADAPT)', 'Обв, % (Модель) (ADAPT)',
                  'ГФ (Модель) (RESTORE)', 'Обв, % (Модель) (RESTORE)']}

temperatures = {'Температура, С':
                ['T устья, С (Модель) (ADAPT)', 'T устья, С (Модель) (RESTORE)', 
                'T прием ЭЦН, C (Модель) (ADAPT)', 'T прием ЭЦН, C (Модель) (RESTORE)']}

pressures_up = {'Давления (Устьевое и буферное), атм':
                ['Рбуф (Ш) (ADAPT)', 'P буф., атм (Модель) (ADAPT)', 'P буф., атм (Модель) (RESTORE)',
             'Рлин ТМ (Ш) (ADAPT)', 'P лин., атм (Модель) (ADAPT)', 'P лин., атм (Модель) (RESTORE)']}

pressures_down = {'Давления (На приеме), атм':
                  ['P прием ЭЦН, атм (Модель) (ADAPT)', 'P прием ЭЦН, атм (Модель) (RESTORE)']}
                 
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш) (ADAPT)', 'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель) (ADAPT)', 'F тока, ГЦ (Модель) (RESTORE)']}

beam_sizes = {'Размер штуцера, мм':
              ['Dшт (Ш) (ADAPT)', 'Dшт (Ш) (RESTORE)']}

powers = {'Активная мощность, Вт':
          ['Активная мощность (СУ) (ADAPT)', 'Мощность, передаваемая СУ (Модель) (ADAPT)',
           'Активная мощность (СУ) (ADAPT) (INTERP)', 'Мощность, передаваемая СУ (Модель) (RESTORE)']}
                       
currents= {'Токи, А':
           ['Ток фазы А (СУ) (ADAPT)', 'I, А (Модель) (ADAPT)', 'I, А (Модель) (RESTORE)']}
               
loads = {'Загрузка, д.ед.':
         ['Загрузка двигателя (Модель) (ADAPT)', 'Загрузка двигателя (Модель) (RESTORE)']}

efficiency = {'КПД ЭЦН, д.ед.':
              ['КПД ЭЦН, д.ед. (Модель) (ADAPT)', 'КПД ЭЦН, д.ед. (Модель) (RESTORE)']}

metrics = {'Метрики расчета':
           ['Значение функции ошибки (Модель) (ADAPT)',
                  'Значение функции ошибки (Модель) (RESTORE)',
          'Относительная ошибка расчетов (N акт), %',
           'Относительная ошибка расчетов (N акт) (INTERP), %']}

dif_pressure = {'Перепад давления в ЭЦН, атм':
                ['Перепад давления в ЭЦН, атм (Модель) (ADAPT)', 'Перепад давления в ЭЦН, атм (Модель) (RESTORE)']}

all_banches = [liquid_rates,essential_mertics, calibrs, metrics, gor_wc, temperatures,pressures_up, 
               pressures_down,frequencies, beam_sizes,powers, currents,loads,efficiency, dif_pressure]

In [ ]:
plot_file_path = f'data/{well_name}/restore_/{well_name}_adapt_and_restore_report.html'
pw.create_report_html(overall_data, all_banches, plot_file_path)

Расчёт различных метрик

In [ ]:
overall_data_with_calibr_gaps = overall_data[overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'] !=
                                            overall_data['К. калибровки по напору - множитель (Модель) (RESTORE)']]
overall_data_with_calibr_gaps = overall_data_with_calibr_gaps.dropna(subset = ['Q ж, м3/сут (Модель) (RESTORE)'])
calibr_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (ADAPT)'],
            overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (RESTORE)'], 'Метрики восстановления Qж с помощью калибровок')
calibr_calc_metrics += 'Средняя относительная ошибка Q ж, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж), %'].abs().mean()) + '\n'
calibr_calc_metrics += 'Средняя относительная ошибка N акт, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт), %'].abs().mean()) + '\n'

interp_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT)'],
            overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT) (INTERP)'], 'Метрики восстановления Qж с помощью интерполяции')
interp_calc_metrics +='Средняя относительная ошибка Q ж (INTERP), %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж) (INTERP), %'].abs().mean()) + '\n' 
interp_calc_metrics +='Средняя относительная ошибка N акт (INTERP), %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт) (INTERP), %'].abs().mean()) + '\n'

In [ ]:
metrics_text_file_path = f'data/{well_name}/restore_/{well_name}__adapt_and_restore_metrics_report.txt'
text_file = open(metrics_text_file_path, "w")
text_file.write(calibr_calc_metrics + '\n' + interp_calc_metrics)
text_file.close()

In [ ]:
tr_name = "Техрежим, , февраль 2019.xls"

tr_file_full_path = os.getcwd() + '\\data\\tr\\' + tr_name
tr_data = prep.read_tr_and_get_data(tr_file_full_path, well_name)
tr_data_df = pd.DataFrame({'Параметры скважины с ТР': list(tr_data.__dict__.values())})
tr_data_df.index = list(tr_data.__dict__.keys())

In [ ]:
calibr_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (ADAPT)'],
            overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (RESTORE)'], 'Метрики восстановления Qж с помощью калибровок', return_df = True )
interp_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT)'],
            overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT) (INTERP)'], 'Метрики восстановления Qж с помощью интерполяции', return_df = True)
overall_metrics = calibr_calc_metrics.append(interp_calc_metrics)
overall_metrics['Средняя относительная ошибка Q ж'] = [overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж), %'].abs().mean(),
                                                      overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж) (INTERP), %'].abs().mean()]
overall_metrics['Средняя относительная ошибка N акт'] = [overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт), %'].abs().mean(),
                                                      overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт) (INTERP), %'].abs().mean()]
overall_metrics['ЭЦН'] = [tr_data.esp_name_str, tr_data.esp_name_str]
overall_metrics['ПЭД'] = [tr_data.motor_name_str, tr_data.motor_name_str]
overall_metrics['Кол-во точек (ADAPT)'] = [len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)']), len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'])]

overall_metrics.index = [well_name + ' (CALIBR)' , well_name + ' (INTERP)']
overall_metrics.to_csv(f'data/{well_name}/restore_/{well_name}_adapt_restore_metrics_tr_report' +  '.csv')
overall_metrics.head()

1. 1354 переделать, не обработался `restore_input`

- 1567 Нет дебита, непонятно почему